## 10分类 10个神经元 Inception Net
Accuracy rate after 10k times training：72.9%

In [1]:
import tensorflow as tf

#### 如果需要使用tf 1.0
tf v2.0 does not have placeholder(), .reset_default_graph().

In [2]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
#读数据：
import os
import pickle as cPickle
import numpy as np

CIFAR_DIR = "../../../other_datasets/cifar-10-batches-py"
print(os.listdir(CIFAR_DIR))

['data_batch_1', '.DS_Store', 'readme.html', 'batches.meta', 'data_batch_2', 'data_batch_5', 'test_batch', 'data_batch_4', 'data_batch_3']


In [4]:
#将数据读取,返回data 和 labels
def load_data(filename):
    """read data from data file."""
    with open(filename, 'rb') as f:
        data = cPickle.load(f, encoding = 'latin1')
        return data['data'], data['labels']

### 实现Inception Net中分组卷积的封装

In [5]:
def inception_block(x,
                    output_channel_for_each_path,
                    name):
    """inception block implementation"""
    """
    Args:
    - x:
    - output_channel_for_each_path: eg. [10, 20, 5]
    - name:
    """
    #1. 这个inception net里面，我们固定分组的第一个是1*1cn，第二个是3*3，第三个是5*5
    #2. 用户可以规定的是每个cn的输出通道数，例如，[10, 20, 5]
    
    #如果定义一个conv1的名字是'conv1',以后就不能再定义'conv1'
    #但是如果在scope里面，就可以在定义一个'conv1',因为此时的名字是scope_name/conv1
    with tf.variable_scope(name): #注意，传入的name在这里被使用：例如后面调用时候的：name = ’inception_3a‘
        conv1_1 = tf.layers.conv2d(x, #注意都是对 原始输入图像进行处理
                                   output_channel_for_each_path[0],
                                   (1,1),
                                    strides = (1,1),
                                    padding = 'same',
                                    activation = tf.nn.relu,
                                    name = 'conv1_1')
        conv3_3 = tf.layers.conv2d(x, #注意都是对 原始输入图像进行处理
                                   output_channel_for_each_path[1],
                                   (3,3),
                                   strides = (1,1),#注意strides是(1,1),如果写成(3,3)就算是padding也没用，输出的长宽会缩小
                                   padding = 'same',
                                   activation = tf.nn.relu,
                                   name = 'conv3_3')
        conv5_5 = tf.layers.conv2d(x, #注意都是对 原始输入图像进行处理
                                   output_channel_for_each_path[2],
                                   (5,5),
                                   strides = (1,1),#注意strides是(1,1),如果写成(5,5)就算是padding也没用
                                   padding = 'same',
                                   activation = tf.nn.relu,
                                   name = 'conv5_5')
        max_pooling = tf.layers.max_pooling2d(x,
                                              (2,2),
                                              (2,2),
                                              name = 'max_pooling')
        
        #现在：
        #conv1_1 的输出长宽和图像长宽一样，但是通道为10
        #conv3_3, 和conv5_5同理，和输出长宽和图像长宽一样
        #所以conv1_1, conv3_3, conv5_5, 可以concatenate，在通道的维度上合并
        #但是max_pooling，因为是(2*2)kernel, 所以输出长宽是原来长宽的1/2
        #因为tf.layers.max_pooling2d() 没有padding的选项，我们要自己padding:
        
        max_pooling_shape = max_pooling.get_shape().as_list()[1:]
        img_shape = x.get_shape().as_list()[1:]
        
        #img_shape: [width, height, channel]
        width_need_padding = (img_shape[0] - max_pooling_shape[0]) // 2
        height_need_padding = (img_shape[1] - max_pooling_shape[1]) // 2
        
        padded_pooling = tf.pad(max_pooling,
                                [[0,0],
                                 [width_need_padding, width_need_padding],
                                 [height_need_padding, height_need_padding], 
                                 [0,0]])
        
        #现在长宽都一样，需要在通道维度上concat，通道是第四个维度，axis = 3
        concat_layer = tf.concat(
        [conv1_1, conv3_3, conv5_5, padded_pooling],
        axis = 3)
        
        return concat_layer        

In [6]:
x = tf.placeholder(tf.float32, [None, 3072])
x_image = tf.reshape(x, [-1, 3, 32, 32])
x_image = tf.transpose(x_image, perm = [0, 2, 3, 1]) #变成[-1, 32, 32, 3]
#x_image： 32 * 32
y = tf.placeholder(tf.int64, [None])

### 开始实现Inception Net

In [7]:
conv1 = tf.layers.conv2d(x_image,
                         32, #output channel number
                         (3,3), #kenel size
                          padding = 'same', #使得输出和我的输入一样大，如果不想padding就padding = 'valid'
                          activation= tf.nn.relu,
                          name = 'conv1' # 输出的时候，看名字我们知道是哪个层，好调试
                         )

pooling1 = tf.layers.max_pooling2d(conv1, # 输入是conv1
                                  (2,2), # kernel size
                                  (2,2), #步长：至少大于kernel size才能达到图像信息压缩
                                   name = 'pool1' # 输出的时候，看名字我们知道是哪个层，好调试
                                  )

inception_2a = inception_block(pooling1,  #注意这里不是pool1，而是pooling1
                               [16, 16, 16],
                               name = 'inception_2a')

inception_2b = inception_block(inception_2a, 
                               [16, 16, 16],
                               name = 'inception_2b')

pooling2 = tf.layers.max_pooling2d(inception_2b,
                                   (2,2),
                                   (2,2),
                                   name = 'pool2')

inception_3a = inception_block(pooling2, 
                               [16, 16, 16],
                               name = 'inception_3a')

inception_3b = inception_block(inception_3a, 
                               [16, 16, 16],
                               name = 'inception_3b')

pooling3 = tf.layers.max_pooling2d(inception_3b,
                                   (2,2),
                                   (2,2),
                                   name = 'pool3')

#将pooling层展开
flatten_pooling3 = tf.layers.flatten(pooling3)

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
Instructions for updating:
Use keras.layers.Flatten instead.


### 全连接层

In [8]:
y_ = tf.layers.dense(flatten_pooling3, 10) #输出10分类
#最后test准确率是：[Test] Step: 10000, acc: 0.72500

Instructions for updating:
Use keras.layers.Dense instead.


In [9]:
loss = tf.losses.sparse_softmax_cross_entropy(labels = y, logits = y_)

In [10]:
#bool:
#predict = p_y_1 > 0.5

#indexes(int):
#第一个维度是同一类别下每个样本，第二维度是同一个样本下的每个类别（一个类别有一个概率值）
predict_indexes = tf.argmax(y_, 1) #1代表第二个维度，求一个样本，哪个类别的概率值最大

#int64:
#predict_float = tf.cast(predict, tf.int64)

#bool:
correct_prediction = tf.equal(predict_indexes, y) #两个int64的array比较

#float64:
correct_prediction_float = tf.cast(correct_prediction, tf.float32)

#float64, 因为correct_prediction_float是[1.0, 0.0, 1.0, ...] 而reduce_mean就是看有多少1，然后除以len
accuracy = tf.reduce_mean(correct_prediction_float)

### AdamOptimizer
AdamOptimizer(learning_rate)：梯度下降的变种，是反向传播算法的变种，初始learning_rate是0.01，目标最小化loss

In [11]:
with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

### cifar10 数据处理
因为mini-batch，所以需要在cifar10上循环遍历数据，每次在sess.run()的feed_dict塞入不同数据

In [12]:
#filenames: 对 训练数据集 和 测试数据集 是分开的
#need_shuffle: 训练数据集需要shuffle，测试数据集不需要shuffle
class CifarData:
    def __init__(self, filenames, need_shuffle):
        #读入数据：
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            #data的对应位置的图像的label，也是label对应位置的label，所以把它们两个zip起来
            all_data.append(data)
            all_labels.append(labels)
            #不再是只调取0，1类 
#             for item, label in zip(data, labels):
#                 if label in [0,1]:
#                     all_data.append(item)
#                     all_labels.append(label)
        #data是numpy的矩阵，
        #item是numpy的向量
        #all_data里面包含了许多numpy的向量，现在我们将这些向量纵向合并到一起，成为一个矩阵：用np.vstack()
        #all_labels里面包含了许多label的向量，现在我们将这些向量横向合并到一起，成为一个矩阵（但是这个矩阵只有一行：n * 1）：用np.hstack()
        self._data = np.vstack(all_data) 
        self._data = self._data / 127.5 - 1 #最后做的改进：将0-255的值二分成0，1
        self._labels = np.hstack(all_labels)
        #print(self._data.shape)
        #print(self._labels.shape)
        self._num_examples = self._data.shape[0] #多少个样本
        self._need_shuffle = need_shuffle
        self._indicator = 0 #遍历到数据集的哪个位置
        if self._need_shuffle:
            self._shuffle_data()
    
    def _shuffle_data(self): #在训练集上shuffle
        # np.random.permutation(6) -> [3,2,5,1,4,0]
        p = np.random.permutation(self._num_examples) #得到一个混的全排
        self._data = self._data[p]
        self._labels = self._labels[p]
        
    def next_batch(self, batch_size): #每次返回batch_size个数个样本
        """return batch_size examples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data() #就算遍历完，可是如果可以shuffle，我们的数据可以复用
                self._indicator = 0
                end_indicator = batch_size + 0
            else:
                raise Exception("have no more examples") #已经遍历完了并且不能shuffle
        if end_indicator > self._num_examples:
            #此时说明我们输入的batch_size比我们的所有样本都要大
            raise Exception("batch size is larger than example size")
        batch_data = self._data[self._indicator : end_indicator]
        batch_labels = self._labels[self._indicator : end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels

### 测试我们的CifarData
1. 首先cifar10拥有50k个样本（10个类别），我们现在是二分类问题，只选取label为0，1的
2. 所以应该有 50k * (1/5) = 10k个样本 (10000, 3072)

In [13]:
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch') ]

train_data = CifarData(train_filenames, True)

In [14]:
test_data = CifarData(test_filenames, False)

In [15]:
batch_data, batch_labels = train_data.next_batch(3)

### 执行流程图

In [18]:
init = tf.global_variables_initializer() #执行初始化
#开启Session，相当于打开绘画
#sess.run()执行计算图
#1. 计算目标:
#表示要训练：[loss, accuracy, train_op]
#表示要测试：[loss, accuracy]

#2. feed_dict:塞入数据
# x: x要塞的数据
# 因为mini-batch，所以需要在cifar10上循环遍历数据，每次塞入不同数据

batch_size = 20 #每次取20张图来 训练 或者 测试
train_steps = 1000
test_steps = 100

with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        #因为train_op返回的东西没有意义，所以用 _ 下划线接住返回值
        loss_val, accu_val, _ = sess.run(
            [loss, accuracy, train_op], 
            feed_dict = {
                x: batch_data, 
                y: batch_labels})  #注意格式！
        #===============================
        #======训练=====================
        if (i+1) % 100 == 0: #使用(i+1)就可以输出500和1000
            print("[Train] Step: %d, loss: %4.5f, acc: %4.5f"\
                 % (i+1, loss_val, accu_val))
        #===============================
        #======测试=====================
        if (i+1) % 500 == 0: #使用(i+1)就可以输出500和1000
            test_data = CifarData(test_filenames, False)
            all_test_acc_vals = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels \
                    = test_data.next_batch(batch_size)
                test_acc_val = sess.run(
                    [accuracy],
                    feed_dict = {
                        x: test_batch_data,
                        y: test_batch_labels
                    })
                all_test_acc_vals.append(test_acc_val)
            test_acc = np.mean(all_test_acc_vals)
            print("[Test] Step: %d, acc: %4.5f" % (i+1, test_acc))

[Train] Step: 100, loss: 1.90456, acc: 0.30000
[Train] Step: 200, loss: 1.61109, acc: 0.50000
[Train] Step: 300, loss: 1.74137, acc: 0.35000
[Train] Step: 400, loss: 1.21318, acc: 0.50000
[Train] Step: 500, loss: 1.59983, acc: 0.40000
[Test] Step: 500, acc: 0.46100
[Train] Step: 600, loss: 1.34017, acc: 0.45000
[Train] Step: 700, loss: 1.30574, acc: 0.60000
[Train] Step: 800, loss: 1.05935, acc: 0.70000
[Train] Step: 900, loss: 0.95500, acc: 0.70000
[Train] Step: 1000, loss: 1.06200, acc: 0.60000
[Test] Step: 1000, acc: 0.56000
